<a href="https://colab.research.google.com/github/raeubaen/fratm/blob/main/sign_plot_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://github.com/MohamedElashri/ROOT/releases/download/ubuntu/root_v6.28.04_Ubuntu_20.04.zip
!unzip /content/root_v6.28.04_Ubuntu_20.04.zip
!apt-get install git dpkg-dev cmake g++ gcc binutils libx11-dev libxpm-dev libxft-dev libxext-dev tar gfortran subversion
!apt-get inatall libpython3.6-dev

# The following is needed because colab upgraded the openssl library
!wget http://archive.ubuntu.com/ubuntu/pool/main/o/openssl/libssl1.1_1.1.1f-1ubuntu2_amd64.deb
!sudo dpkg -i libssl1.1_1.1.1f-1ubuntu2_amd64.deb

import sys
sys.path.append("/content/root_build/")
sys.path.append("/content/root_build/bin/")
sys.path.append("/content/root_build/include/")
sys.path.append("/content/root_build/lib/")
import ctypes
ctypes.cdll.LoadLibrary('/content/root_build/lib//libCore.so')
ctypes.cdll.LoadLibrary('/content/root_build/lib//libThread.so')
ctypes.cdll.LoadLibrary('/content/root_build/lib//libTreePlayer.so')


Streaming output truncated to the last 5000 lines.
  inflating: root_build/tutorials/gui/WorldMap.C  
  inflating: root_build/tutorials/gui/guitest.C  
  inflating: root_build/tutorials/gui/buttonChangelabel.C  
  inflating: root_build/tutorials/gui/exec_macro.C  
  inflating: root_build/tutorials/gui/buttonsLayout.C  
  inflating: root_build/tutorials/gui/statusBar.C  
  inflating: root_build/tutorials/gui/buttongroupState.C  
  inflating: root_build/tutorials/gui/exec3.C  
   creating: root_build/tutorials/physics/
  inflating: root_build/tutorials/physics/PhaseSpace.C  
   creating: root_build/tutorials/math/
  inflating: root_build/tutorials/math/Bessel.py  
  inflating: root_build/tutorials/math/principal.py  
  inflating: root_build/tutorials/math/mathcoreStatFunc.C  
  inflating: root_build/tutorials/math/exampleMultiRoot.C  
  inflating: root_build/tutorials/math/mathcoreGenVector.C  
  inflating: root_build/tutorials/math/principal.C  
  inflating: root_build/tutorials/math/ex

<CDLL '/content/root_build/lib//libTreePlayer.so', handle 567ef81ac9c0 at 0x7e6f7d10f670>

In [6]:
!rm -r fratm

In [7]:
!git clone https://github.com/raeubaen/fratm.git

Cloning into 'fratm'...
remote: Enumerating objects: 192, done.
remote: Counting objects: 100% (192/192), done.
remote: Compressing objects: 100% (151/151), done.
remote: Total 192 (delta 94), reused 119 (delta 38), pack-reused 0
Receiving objects: 100% (192/192), 15.65 MiB | 22.90 MiB/s, done.
Resolving deltas: 100% (94/94), done.


In [8]:
folder = "fratm"

In [9]:
!python3 fratm/sign.py > sign_1mese.csv

[100, 7000, 76.4679773175, 0.025, ['1: 0.0', '2: 0.0', '3: 0.0', '4: 0.1', '5: 2.0', '6: 6.6', '7: 10.1', '8: 11.0', '9: 10.2', '10: 8.6', '11: 6.9', '12: 5.3', '13: 4.0', '14: 2.9', '15: 2.2', '16: 1.6', '17: 1.2', '18: 0.9', '19: 0.7', '20: 0.5', '21: 0.4', '22: 0.3', '23: 0.2', '24: 0.2', '25: 0.2', '26: 0.1', '27: 0.1', '28: 0.1', '29: 0.1']]
[150, 7000, 76.4679773175, 0.025, ['1: 0.0', '2: 0.0', '3: 0.0', '4: 0.1', '5: 2.0', '6: 6.6', '7: 10.1', '8: 11.0', '9: 10.2', '10: 8.6', '11: 6.9', '12: 5.3', '13: 4.0', '14: 2.9', '15: 2.2', '16: 1.6', '17: 1.2', '18: 0.9', '19: 0.7', '20: 0.5', '21: 0.4', '22: 0.3', '23: 0.2', '24: 0.2', '25: 0.2', '26: 0.1', '27: 0.1', '28: 0.1', '29: 0.1']]
[200, 7000, 76.4679773175, 0.025, ['1: 0.0', '2: 0.0', '3: 0.0', '4: 0.1', '5: 2.0', '6: 6.6', '7: 10.1', '8: 11.0', '9: 10.2', '10: 8.6', '11: 6.9', '12: 5.3', '13: 4.0', '14: 2.9', '15: 2.2', '16: 1.6', '17: 1.2', '18: 0.9', '19: 0.7', '20: 0.5', '21: 0.4', '22: 0.3', '23: 0.2', '24: 0.2', '25: 0.2'

In [21]:
import ROOT
from ROOT import *
import numpy as np
import argparse
import pandas as pd
import os
%jsroot on

ROOT.gROOT.LoadMacro(f'{folder}/rootlogon_sign.C')

def GetH(tree, val, range="", cut="", opt="", hname="h_temp"):
    hpar = "" if (range=="") else "(" + range + ")"
    hval = val + " >> " + hname + hpar
    tree.Draw(hval, cut, opt)
    return gDirectory.Get(hname)


def FormatH(obj, title, x_label, y_label, stats=1):
    obj.SetStats(stats)
    obj.SetTitle(title)
    obj.GetXaxis().SetTitle(x_label)
    obj.GetYaxis().SetTitle(y_label)


class Canva():

    szdef = [900,600]

    def __init__(self) -> None:
        self.cnt = 0
        self.canvases = []

    def new(self, s1 = szdef[0], s2 =szdef[1]):
        n = "c_" + str(self.cnt)
        self.canvases.append(TCanvas(n, n, s1, s2))
        self.canvases[self.cnt].Draw()
        # self.canvases[self.cnt].SetLeftMargin(0.13)
        # self.canvases[self.cnt].SetRightMargin(0.07)
        # self.canvases[self.cnt].SetTopMargin(0.07)
        # self.canvases[self.cnt].SetBottomMargin(0.19)

        self.cnt += 1
        return self.canvases[self.cnt-1]

    def divide(self, rows, columns):
        self.canvases[self.cnt-1].Divide(rows,columns)

    def cd(self, i):
        self.canvases[self.cnt-1].cd(i)

    def update(self):
        self.canvases[self.cnt-1].Update()


    def savelast(self, titl):
        self.canvases[self.cnt-1].SaveAs(titl)


def FillGraph(gra: TGraphErrors, x, y, ex, ey):
    gra.AddPoint(x, y)
    n = gra.GetN()
    gra.SetPointError(n-1, ex, ey)
    return n



filein = "sign_1mese"
df = pd.read_csv(f"{filein}.csv", skiprows=1, sep=",")
df.columns = ["reso", "cut", "s", "b"]
mmscale = 1000 #put 1 for micron

c = Canva()
graCut = TGraphErrors("graCut") #FARLI Th2 in caso
graS = TGraphErrors("graS")
graB = TGraphErrors("graB")
FormatH(graCut, " ", "Resolution on vertex x position [#mum]", "Min. cut for 5#sigma at 95% prob [#mum]", stats=1) if mmscale ==1 else FormatH(graCut, " ", "Resolution on vertex x position [mm]", "Min. cut for 5#sigma at 95% prob [mm]", stats=1)


for i in range(0,len(df["reso"])-1):
    FillGraph(graCut, df["reso"][i]/mmscale,df["cut"][i]/mmscale , 0, 0)
    FillGraph(graS, df["reso"][i]/mmscale,df["s"][i] , 0, 0)
    FillGraph(graB, df["reso"][i]/mmscale,df["b"][i] , 0, 0)




ROOT.gStyle.SetOptStat(0)
# Create the canvas
can = ROOT.TCanvas('can', '', 1200, 700)
can.Draw()


leg = TLegend(.32,.65,.65,.85)
leg.SetBorderSize(0)
leg.SetFillColor(0)
leg.SetFillStyle(0)
leg.SetMargin(0.12)
#leg.SetTextSize(1)

#x_range = [10, 70]

# Use a base pad (the canvas) and then a transparent pad
clear_pad = ROOT.TPad('clear_pad', '', 0, 0, 1, 1)
clear_pad.SetFillColor(0)
clear_pad.SetFillStyle(4000)
clear_pad.SetFrameFillStyle(0)


clear_pad1 = ROOT.TPad('clear_pad1', '', 0, 0, 1, 1)
clear_pad1.SetFillColor(0)
clear_pad1.SetFillStyle(4000)
clear_pad1.SetFrameFillStyle(0)

# Turn off the tick marks on the right hand side y-axis
can.SetTicky(0)

h1 = graCut.Clone()
leg.AddEntry(h1, "Min. cut for 5#sigma at 95% probability" , "P")
h1.SetLineColor(ROOT.kBlack)
h1.SetMarkerColor(ROOT.kBlack)
h1.SetMarkerStyle(ROOT.kFullCircle)



h1.Draw('AP')


# Draw h2 information
clear_pad.Draw()
clear_pad.cd()
clear_pad.SetTicks(0,0)
# h2 should be on a log scale
clear_pad.SetLogy(1)
h2 = graB.Clone()

h2.GetYaxis().SetLabelColor(ROOT.kBlue+1)
h2.GetYaxis().SetLabelFont(42)

h2.GetYaxis().SetTitle("Exp. events")
h2.GetYaxis().SetTitleColor(ROOT.kBlue+1)
h2.GetYaxis().SetTitleFont(62)
h2.GetYaxis().SetTitleOffset(1.4)
h2.GetYaxis().SetLabelSize(0.045)


h2.GetYaxis().SetLabelColor(ROOT.kBlue+1)
h2.GetYaxis().SetAxisColor(ROOT.kBlue+1)
h2_y_min = 0.5e-2#*mmscale
h2_y_max = 1.5e4#*mmscale
h2.GetYaxis().SetRangeUser(h2_y_min, h2_y_max)
h2.SetLineColor(ROOT.kBlue+1)
h2.SetMarkerColor(ROOT.kBlue+1)
h2.SetMarkerStyle(33)
h2.SetMarkerSize(1.5)

leg.AddEntry(h2, "Signal exp. events", "P" )

h2.Draw('APY+')

clear_pad1.Draw()
clear_pad1.cd()
clear_pad1.SetTicks(0,0)
clear_pad1.SetLogy(1)
h3 = graS.Clone()
h2_y_min = 0.5e-2#*mmscale
h2_y_max = 1.5e4#*mmscale
h3.GetYaxis().SetRangeUser(h2_y_min, h2_y_max)
h3.SetLineColor(ROOT.kBlue+1)
h3.SetMarkerColor(ROOT.kBlue+1)
h3.SetMarkerStyle(29)

h3.GetYaxis().SetTitleColor(ROOT.kBlue+1)
h3.GetYaxis().SetTitleFont(62)
h3.GetYaxis().SetTitleOffset(1.3)
h3.GetYaxis().SetLabelSize(0.045)


h3.GetYaxis().SetLabelColor(ROOT.kBlue+1)
h3.GetYaxis().SetAxisColor(ROOT.kBlue+1)
h2_y_min = 0.5e-2#*mmscale
h2_y_max = 1.5e4#*mmscale
h3.GetYaxis().SetRangeUser(h2_y_min, h2_y_max)
h3.SetLineColor(ROOT.kBlue+1)
h3.SetMarkerColor(ROOT.kBlue+1)
#h3.SetMarkerStyle(33)
h3.SetMarkerSize(1.3)



h3.Draw('APY+')


leg.AddEntry(h3, "Background exp. events", "P" )
leg.Draw()

can.SaveAs(f"{filein}.pdf")
can.SaveAs(f"{filein}.C")



 Beginning new ROOT session with private TStyle 


Error in <TGraphErrors::TGraphErrors>: Cannot open file: graCut, TGraphErrors is Zombie
Error in <TGraphErrors::TGraphErrors>: Cannot open file: graS, TGraphErrors is Zombie
Error in <TGraphErrors::TGraphErrors>: Cannot open file: graB, TGraphErrors is Zombie
Warning in <TCanvas::Constructor>: Deleting canvas with same name: can
Info in <TCanvas::Print>: pdf file sign_1mese.pdf has been created
Info in <TCanvas::SaveSource>: C++ Macro file: sign_1mese.C has been generated
